In [1]:
# 주소 불러오기
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

html=urlopen('https://www.weather.go.kr/w/dust/dust-obs-days.do')
bs=BeautifulSoup(html.read(), 'html.parser')

select = bs.find('select',{'id':'select-stn'})

c=[]
for i in select.find_all('option'):
    c.append(i.text)

uu=[]
for i in c:
    if ('[' not in i) and (i[0] != ' '):
        uu.append(int(i.split(' ')[0]))

In [2]:
# 지역명 처리??
kk=[]
for i in c:
    if ('[' not in i) and (i[0] != ' '):
        kk.append(i.split(' ')[1])
        
addr = {'서울':'서울특별시','백령도':'백령도','파주(':'파주시',
        '동두천':'동두천시','인천':'인천광역시','수원':'수원시',
        '강화':'강화군','양평':'양평군','이천':'이천시',
        '강릉':'강릉시','북강릉':'북강릉','북춘천':'북춘천',
        '춘천':'춘천시','원주':'원주시','대관령':'대관령',
        '속초':'속초시','동해':'동해시','철원':'철원군',
        '영월':'영월군','인제':'인제군','홍천':'홍천군',
        '태백':'태백시','충주':'충주시','청주':'청주시',
        '추풍령':'추풍령면','제천':'제천시','보은':'보은군',
        '대전':'대전광역시','서산':'서산시','홍성':'홍성군',
        '천안':'천안시','보령':'보령시','부여':'부여군',
        '금산':'금산군','군산':'군산시','전주':'전주시',
        '부안':'부안군','임실':'임실군','정읍':'정읍시',
        '남원':'남원시','장수':'장수군','고창':'고창군',
        '광주':'광주광역시','목포':'목포시','여수':'여수시',
        '완도':'완도군','진도':'진도군','흑산도':'흑산도',
        '순천':'순천시','장흥':'장흥군','해남':'해남군',
        '고흥':'고흥군','대구(구)':'대구광역시(구)',
        '대구(기)':'대구광역시(기)','울진':'울진군','안동':'안동시',
        '상주':'상주시','포항':'포항시','울릉도':'울릉군',
        '봉화':'봉화군','영주':'영주시','문경':'문경시',
        '영덕':'영덕군','의성':'의성군',
        '구미':'구미시','영천':'영천시','부산':'부산광역시',
        '울산':'울산광역시','창원(':'창원시','통영':'통영시',
        '진주':'진주시','거창':'거창군','합천':'합천군',
        '밀양':'밀양시','산청':'산청군','거제':'거제시','남해':'남해군',
        '제주':'제주특별자치도','고산':'고산리',
        '서귀포':'서귀포시','성산(':'성산읍','(구)성산포':'성산읍(구)'}
kk = pd.Series(kk)

kk = kk.apply(lambda v: addr.get(v, v))

In [3]:
# 행정구역 설정
xx=['서울특별시']
aa=['백령도','파주시','동두천시','인천광역시','수원시','강화군','양평군','이천시']
bb=['강릉시','북강릉','북춘천','춘천시','원주시','대관령','속초시','동해시',
    '철원군','영월군','인제군','홍천군','태백시']
cc=['충주시','청주시','추풍령면','제천시','보은군']
dd=['대전광역시','서산시','홍성군','천안시','보령시','부여군','금산군']
ee=['군산시','전주시','부안군','임실군','정읍시','남원시','장수군','고창군']
ff=['광주광역시','목포시','여수시','완도군','진도군','흑산도','순천시','장흥군','해남군','고흥군']
gg=['대구광역시(구)','대구광역시(기)','울진군','안동시','상주시','포항시',
    '울릉군','봉화군','영주시','문경시','영덕군','의성군','구미시','영천시']
hh=['부산광역시','울산광역시','창원시','통영시','진주시','거창군',
    '합천군','밀양시','산청군','거제시','남해군']
ii=['제주특별자치도','고산리','서귀포시','성산읍','성산읍(구)']

In [5]:
# 해당 페이지 데이터 프레임 
# 마지막((구)성산포)은 데이터가 없으므로 제외하기 !!
dfdf0 = list()
dfdf1 = list()
dfdf2 = list()
dfdf3 = list()
dfdf4 = list()
dfdf5 = list()
dfdf6 = list()
dfdf7 = list()


for file in uu[:81]:
    html1=urlopen('https://www.weather.go.kr/w/dust/dust-obs-days.do?type=1&stnId='+str(file))
    bs1=BeautifulSoup(html1.read(), 'html.parser')

    table = bs1.find('table')

    a=[]
    for i in table.find_all('tr'):
        a.append(i.text.strip())

    b=[pd.Series(a[0].split('\n'))]
    for j in range(len(a)-1, 0, -1):
        b.append(pd.Series(a[j].split('\n')))


    DF = pd.concat(b, axis = 1).T

    DF = DF.replace('.', 0)

    day = list()
    for i in DF.iloc[1:, 0]:
        for j in DF.iloc[0, 1:]:
            day.append(i[:-1] + '-' + j[:-1])
    day = pd.Series(day)

    value=[]
    for i in range(1, DF.shape[0]):
        value.append(DF.iloc[i, 1:])

    values = pd.concat(value)
    values = values.astype(int)


    df = day.to_frame()
    df['1'] = list(values)

    df[0] = pd.to_datetime(df[0])

    month = list()
    for i in df[0]:
        month.append(i.to_period('m'))
    df[0]=month

    df.columns=['날짜', '데이터']
    
    df.insert(1, '지역', kk[uu.index(file)])
    
    if df['지역'][0] in xx:
        df.insert(2, '행정구역', '서울특별시')
    elif df['지역'][0] in aa:
        df.insert(2, '행정구역', '경기도')
    elif df['지역'][0] in bb:
        df.insert(2, '행정구역', '강원도')
    elif df['지역'][0] in cc:
        df.insert(2, '행정구역', '충청북도')
    elif df['지역'][0] in dd:
        df.insert(2, '행정구역', '충청남도')
    elif df['지역'][0] in ee:
        df.insert(2, '행정구역', '전라북도')
    elif df['지역'][0] in ff:
        df.insert(2, '행정구역', '전라남도')
    elif df['지역'][0] in gg:
        df.insert(2, '행정구역', '경상북도')
    elif df['지역'][0] in hh:
        df.insert(2, '행정구역', '경상남도')
    elif df['지역'][0] in ii:
        df.insert(2, '행정구역', '제주특별자치도')
    
    dfdf0.append(df)
#     df.to_csv('./yellow/'+str(kk[uu.index(file)])+'.csv')

In [ ]:
# # 하나씩 csv 파일로 내보내기
# x0=pd.concat(dfdf0)
# x0.to_csv('./xxx/x0.csv')

# x1=pd.concat(dfdf1)
# x1.to_csv('./xxx/x1.csv')

# x2=pd.concat(dfdf2)
# x2.to_csv('./xxx/x2.csv')

# x3=pd.concat(dfdf3)
# x3.to_csv('./xxx/x3.csv')

# x4=pd.concat(dfdf4)
# x4.to_csv('./xxx/x4.csv')

# x5=pd.concat(dfdf5)
# x5.to_csv('./xxx/x5.csv')

# x6=pd.concat(dfdf6)
# x6.to_csv('./xxx/x6.csv')

# x7=pd.concat(dfdf7)
# x7.to_csv('./xxx/x7.csv')

In [ ]:
# # 파일 읽어들이기
# p0=pd.read_csv('./xxx/x0.csv')
# p1=pd.read_csv('./xxx/x1.csv')
# p2=pd.read_csv('./xxx/x2.csv')
# p3=pd.read_csv('./xxx/x3.csv')
# p4=pd.read_csv('./xxx/x4.csv')
# p5=pd.read_csv('./xxx/x5.csv')
# p6=pd.read_csv('./xxx/x6.csv')
# p7=pd.read_csv('./xxx/x7.csv')

In [ ]:
# # 하나의 최종 데이터프레임 만들기
# DF_ = pd.concat([p0, p1, p2, p3, p4, p5, p6, p7])
# DF_.columns

In [ ]:
# # 필요 없는 컬럼 지우기
# DF_=DF_.drop(['Unnamed: 0'], axis = 1)

In [ ]:
# # 날짜 컬럼 형변환
# DF_['날짜'] = pd.to_datetime(DF_['날짜'])

# months = list()
# for i in DF_['날짜']:
#     months.append(i.to_period('m'))
    
# DF_['날짜'] = months

In [ ]:
# # 파일 정보 확인
# DF_.info()

In [ ]:
# DF_.to_csv('result1.csv')

In [ ]:
##################################################

In [6]:
df_leg = pd.concat(dfdf0)
df_leg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45936 entries, 0 to 623
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype    
---  ------  --------------  -----    
 0   날짜      45936 non-null  period[M]
 1   지역      45936 non-null  object   
 2   행정구역    45936 non-null  object   
 3   데이터     45936 non-null  int64    
dtypes: int64(1), object(2), period[M](1)
memory usage: 1.8+ MB


In [9]:
df_leg.columns = ['날짜', '지역', '행정구역', '황사관측일수']

In [10]:
df_leg.to_csv('황사관측일수.csv')